In [ ]:
# MIV value map

#Считаем мив для все эксп картин по одной теор картине
import numpy as np
from PIL import Image
import h5py
import time
import matplotlib.pyplot as plt

#timer on
begin_time = time.time()

time.sleep(1)

#enter path to h5 file name
filename = 'InputH5FileName'
f = h5py.File(filename, 'r')
#pick corret 
group = f['EMData/EBSD/EBSDPatterns']
Ii = group[0]


MIV_list = []
count=10

name='mask for images' 
#Считаем MIVы для каждого изображения из базы
for i in range(1,10001):
    if i==count:
        count=count*10
        name=name[0:-1]
    Ij = np.asarray(Image.open(name+str(i)+'.jpg'), dtype='uint8')
    
    dif_abs = abs(Ii/np.max(Ii)-Ij/np.max(Ij))
    MIV = np.sum(dif_abs)/(Ij.shape[0]*Ij.shape[1])
    MIV_list.append(MIV)
    
end_time = time.time()
print('Calculation time = ', end_time - begin_time)

miv_mas = np.array(MIV_list).reshape(100, 100)

plt.imshow(miv_mas)

In [ ]:
#ploting difference map
import h5py
import numpy as np
import matplotlib.pyplot as plt

#open input image as array
arr1 = np.asarray(Image.open('image file name'), dtype=np.uint8)
arr2 = np.asarray(Image.open('image file name'), dtype=np.uint8)

#programm can also use h5 file as input image
# f = h5py.File('name of h5 file', 'r')
# arr2 = f['EMData/EBSD/EBSDPatterns'][0]

#pixel subtraction
dif_arr = abs(arr1 - arr2)

#ploating map with colorbar
plt.imshow(mas, cmap = 'hot')
plt.colorbar()
plt.show()

#max value is 255 (worse), min value - 0 (better)
print(np.sum(mas)/(mas.shape[0]*mas.shape[1]))

In [ ]:
#NCC between two objects

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image


#functions used in indexing (functions created by A. Winkelmann)
def norm_data(data):
    mean_data=np.mean(data)
    std_data=np.std(data, ddof=1)
    #return (data-mean_data)/(std_data*np.sqrt(data.size-1))
    return (data-mean_data)/(std_data)

def ncc(data0, data1):
    return (1.0/(data0.size-1)) * np.sum(norm_data(data0)*norm_data(data1))

#open experimental input image as array
exp = np.array(Image.open('InputImage.jpg').convert(mode='L'), dtype=np.float64)
#open simulated input image as array
sim = np.array(Image.open('InputImage.png').convert(mode='L'), dtype=np.float64)
#programm can also use h5 file as input image
# f = h5py.File('name of h5 file', 'r')
# sim = f['EMData/EBSD/EBSDPatterns'][0]

#printing ncc value between two images                   
print('Max NCC = ', ncc(exp, sim))

#ploting exp and sim images
fig, axs = plt.subplots( 2, figsize=(9,6))

im00 = axs[0].imshow(test_sim, cmap=plt.cm.gray)
im01 = axs[1].imshow(test_exp, cmap=plt.cm.gray)

fig.colorbar(im00, ax=axs[0])
fig.colorbar(im01, ax=axs[1])

In [ ]:
#indexing based on metric NCC
import numpy as np
import h5py
from PIL import Image
import time

#functions used in indexing (functions created by A. Winkelmann)
def norm_data(data):
    mean_data=np.mean(data)
    std_data=np.std(data, ddof=1)
    #return (data-mean_data)/(std_data*np.sqrt(data.size-1))
    return (data-mean_data)/(std_data)

def ncc(data0, data1):
    return (1.0/(data0.size-1)) * np.sum(norm_data(data0)*norm_data(data1))

#timer on
begin_time = time.time()
time.sleep(1)

#enter path to file
f = h5py.File('name of h5 file', 'r')

#loading data from file in a variable
group = f['EMData/EBSD/EBSDPatterns']

#create lists for data storage 
max_NCC=[]
NCC_list=[]
index=[]

#second attribute in range is a number of experimental images
for j in range(1,10001):
    #enter the path to image
    exp = np.array(Image.open('mask for images'+str(j)+'.jpg').convert(mode='L'), dtype=np.float64)
    for i in range(0,group.shape[0]):    
        #got data from h5 file with index i
        sim = group[i]
        #use the function ncc to experimental and simulated data
        ncc_0  = ncc(exp, sim)
        #add ncc value in  template list
        NCC_list.append(ncc_0)
    #find max ncc value in template list and add this value in another list
    max_NCC.append(max(NCC_list))
    #add index of the max value in list
    index.append(NCC_list.index(max(NCC_list)))
    #reset the template list
    NCC_list=[]

                        
#timer off
end_time = time.time()
print('Calculation time = ', end_time - begin_time)